In [22]:
import mlflow
from mlflow.models import ModelSignature
from mlflow.types.schema import Schema, ColSpec, ParamSchema, ParamSpec
import requests

class MyModel(mlflow.pyfunc.PythonModel):
    def predict(self, ctx, model_input, params):
        flower_name = model_input['flowerName'][0]
        return {"flowerName":flower_name.upper() if params['do_work'] else flower_name, "do_work": params['do_work']  }

In [23]:
params_schema = ParamSchema(
    [
        ParamSpec("do_work", "boolean", False)
    ]
)

input_schema = Schema(
    [
        ColSpec("string", "flowerName")
    ]
)

output_schema = Schema(
    [
        ColSpec("string","flowerName")
    ]
)

signature = ModelSignature(inputs=input_schema, outputs=output_schema, params = params_schema)

mlflow.set_experiment("PyFunc")
with mlflow.start_run() as run:
    model_info = mlflow.pyfunc.log_model(
        python_model=MyModel(), artifact_path="my_model", signature=signature,
    )
    
    mlflow.register_model(model_uri = f"runs:/{run.info.run_id}/my_model", name="my_model")

Registered model 'my_model' already exists. Creating a new version of this model...
2023/12/13 20:16:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: my_model, version 9
Created version '9' of model 'my_model'.


# Usage

In the following are two ways to use the registered models. 

## Loading the model

Using pyfunc.load_model you can test if your model is working well, but it doesn`t involves nothing of the model services feature.

In order to use it, change the version of model_uri to the latests version you deployed

In [26]:
model = mlflow.pyfunc.load_model(model_uri=f"models:/my_model/9")
print(model.predict({"flowerName":"rose"}, params={"do_work":True}))
print(model.predict({"flowerName":"rose"}, params={"do_work":False}))

{'flowerName': 'ROSE', 'do_work': True}
{'flowerName': 'rose', 'do_work': False}


## Model services

Go to the published services page and create a model, run it and wait until the running state. Change the URL field below to the url of the published services.

In [20]:
# Model service 🤘

url = "https://10.137.137.218:5000"+"/invocations"
data = {
    "inputs":{
        "flowerName":["sunflower"],
    },
    "params":{
        "do_work": True
    }
}

headers = {'Content-type': 'application/json'}
x = requests.post(url, json = data, verify=False, headers=headers)

print(x.text)

{"predictions": {"flowerName": "SUNFLOWER", "do_work": true}}


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host '10.137.137.218'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
